# Restrictor calculator for Alex Clarke's 3d printed ventilator restrictors

Dr Daniel Duke<br>
Laboratory for Turbulence Research in Aerospace & Combustion (LTRAC)<br>
Department of Mechanical & Aerospace Engineering<br>
Monash University<br>

Last Update: 7th July, 2020

Enquiries: daniel.duke@monash.edu

## General information

This calculator is designed for use with Alex Clarke's 3D printed ventilator restrictor designs:
https://www.prusaprinters.org/prints/25808-3d-printed-circuit-splitter-and-flow-restriction-d

The calculator's parameters were determined by empirical least squares fitting to experimental and simulated flow data. It is valid within the following range of parameters:

Diameter $D$: 0 - 2 mm<br>
Flow Rate $Q$: 0 - 60 L/min<br>
Pressure drop $\Delta P$: 0 - 100 cm H2O<br>

## Citation

This calculator is based on the following research article. If you use this resoure in your research, please cite the following:

< CITATION AND LINK TO PREPRINT GOES HERE >

## How to Use
1. Set any two parameters (diameter, flow rate or pressure drop)
2. Set the parameter that you want to find to zero.
3. Click the Calculate button. The missing parameter will be updated.
4. To repeat the calculation, you will need to set the slider for the unknown parameter back to zero again.

In [ ]:
from ipywidgets import widgets
from IPython.display import display
import numpy as np
from scipy.optimize import fsolve
import sys

dP_input = widgets.FloatSlider(value=0,min=0,max=200,step=1,description="$\Delta P$: cm H$_2$O")
Q_input = widgets.FloatSlider(value=0,min=0,max=100,step=0.1,description="$Q$: L/min")
D_input = widgets.FloatSlider(value=0,min=0,max=5,step=0.1,description="$D$: mm")
button  = widgets.Button(description="Calculate")

def solve_dP(Q,D):
    D = np.abs(D) # enforce positive diameter value
    return 3.55039*(D**-4.02667)*(Q**2) + 10.2758*(D**-3.46302)*Q

def solve_Q(dP,D):
    fwrap = lambda q: solve_dP(q,D) - dP
    x = fsolve(fwrap, x0=0)
    return x

def solve_D(dP,Q):
    fwrap = lambda q: solve_dP(Q,q) - dP
    x = fsolve(fwrap, x0=2.0) # start with nonzero estimate
    return x

def run_calculation(b):
    dP, Q, D = dP_input.value, Q_input.value, D_input.value
    
    # Solve for pressure drop
    if (dP==0) and (Q!=0) and (D!=0):
        dP_out = solve_dP(Q,D)
        print("Q = %0.2f L/min, D = %0.2f mm:  Pressure drop = %0.2f cm H2O" % (Q,D,dP_out))
        dP_input.value = dP_out
        
    # Solve for flow rate
    elif (dP!=0) and (Q==0) and (D!=0):
        Q_out = solve_Q(dP,D)
        print("dP = %0.2f cm H2O, D = %0.2f mm:  Flow rate = %0.2f L/min" % (dP,D,Q_out))
        Q_input.value = Q_out
    
    # Solve for diameter
    elif (dP!=0) and (Q!=0) and (D==0):
        D_out = solve_D(dP,Q)
        print("dP = %0.2f cm H2O, Q = %0.2f L/min:  Restrictor diameter = %0.2f mm" % (dP,Q,D_out))
        D_input.value = D_out
        
    else:
        print("Cannot solve - please set one value to be zero.")
    
    return
        
button.on_click(run_calculation)

display(dP_input)
display(Q_input)
display(D_input)
print("Set the parameter you want to find to zero and click:")
display(button);